# Day 1 - Exercise

1. Query all sessions of ***rat*** with ***axona*** recording
2. Query all recordings ingested from the directory at ***valentno/Data/RatLauren/20171123/1***
3. Every unit has a unit type, ***ClusterType*** - for animal named "24321", for each cluster type, list all the units of that type
4. For animal named "24321", pick one session, fetch the x, y position of the tracked rigid body of that animal. Bonus: plot it
5. For the same animal/session in (4), from each probe, identify the unit with highest spike count
6. For those 2 units from (5), fetch the spike times, x, and y positions associated with each spike
7. Select any ***neurologger*** recording, fetch the LFP at each tetrode

In [ ]:
import datajoint as dj
dj.config['display.limit'] = 10
dj.config['database.host'] = 'datajoint.it.ntnu.no'
dj.conn()

In [ ]:
animal = dj.create_virtual_module('animal', 'prod_mlims_data')
reference = dj.create_virtual_module('reference', 'group_shared_reference')
acquisition = dj.create_virtual_module('acquisition', 'group_shared_acquisition')
tracking = dj.create_virtual_module('tracking', 'group_shared_tracking')
behavior = dj.create_virtual_module('behavior', 'group_shared_behavior')
ephys = dj.create_virtual_module('ephys', 'group_shared_ephys')
analysis = dj.create_virtual_module('analysis', 'group_shared_analysis')
ingestion = dj.create_virtual_module('ingestion', 'group_shared_ingestion')

## 1) Query all sessions of ***rat*** with ***axona*** recording

In [13]:
acquisition.Session & 'datasource_id = 0'

animal_id,datasource_id,session_time start time of this session,session_note
00342c6d86e97d43,0,2019-07-04 20:35:12,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,=BLOB=
61fd2ac184c13c73,0,2017-10-09 16:07:29,=BLOB=
61fd2ac184c13c73,0,2017-10-13 10:32:43,=BLOB=


In [14]:
animal.Animal()

animal_id,datasource_id,animal_species,animal_name,animal_sex,animal_dob,color
0000da5d6810d149,0,mouse,85106,F,2018-10-07,None
0000df481fc1421d,0,rat,20399,F,2014-08-06,None
00016d089675e2f2,0,mouse,55885,M,2016-02-12,None
0003a07550faa59b,0,mouse,78224,F,2017-11-30,None
0005070177484ee2,0,rat,20259,M,2014-07-21,None


In [10]:
rat = animal.Animal & 'animal_species = "rat"'

In [15]:
axona = acquisition.Recording & 'recording_system = "axona"'

In [26]:
acquisition.Session & rat & axona

animal_id,datasource_id,session_time start time of this session,session_note
b013c9113d60f3e6,0,2019-09-03 13:01:53,=BLOB=


## 2) Query all recordings ingested from the directory at ***valentno/Data/RatLauren/20171123/1***

In [29]:
acquisition.Recording & (acquisition.Session.Directories & 'input_dir = "valentno/Data/RatLauren/20171123/1"')

animal_id,datasource_id,session_time start time of this session,insertion_time When this probe was inserted,recording_time start time of this recording,recording_system,"probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),recording_order the ordering of this recording in this session,recording_duration (s) duration of this recording,recording_name name of this recording (e.g. 27032019laserSess1)
ca13934830a1af36,0,2017-11-23 15:58:00,2017-02-07 15:12:29,2017-11-23 15:58:00,neurologger,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,0,3335.0,s1


In [91]:
acquisition.Session & 'session_time LIKE "%RatLauren/20171123/1%"'

C:\Users\thinh\AppData\Roaming\Python\Python37\site-packages\datajoint\connection.py:212: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


animal_id,datasource_id,session_time start time of this session,session_note


## 3) Every unit has a unit type, ***ClusterType*** - for animal named "24321", for each cluster type, list all the units of that type

In [47]:
ephys.Unit()

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),unit,electrode_group electrode group,electrode electrode,cluster_type
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,0,0,1,noise
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,1,0,1,noise
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,2,0,1,noise
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,3,0,3,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,4,0,2,good


In [50]:
ephys.Unit.describe();

-> ephys.CuratedClustering
unit                 : smallint                     
---
-> reference.ElectrodeConfig.Electrode
-> reference.ClusterType



In [83]:
ephys.Unit()

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),unit,electrode_group electrode group,electrode electrode,cluster_type
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,0,0,1,noise
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,1,0,1,noise
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,2,0,1,noise
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,3,0,3,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,4,0,2,good


In [87]:
ephys.Unit & f'cluster_type = "{cluster_types[0]}"'

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),unit,electrode_group electrode group,electrode electrode,cluster_type
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,3,0,3,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,4,0,2,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,8,0,3,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,9,0,5,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,10,0,6,good


In [88]:
ephys.Unit & {'cluster_type': cluster_types[0]}

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),unit,electrode_group electrode group,electrode electrode,cluster_type
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,3,0,3,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,4,0,2,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,8,0,3,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,9,0,5,good
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,10,0,6,good


In [54]:
for cluster_type in cluster_types:
    print(ephys.Unit & {'cluster_type': cluster_type})

*animal_id     *datasource_id *insertion_tim *group_name    *time_window_i *cluster_metho *cluster_param *probe_model   *electrode_con *sync_method   *curation_time *unit    electrode_grou electrode     cluster_type  
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------+ +------------+ +-----------+ +------------+
40bfa03a7a7018 0              2019-09-11 14: det_144c54937f a1c6a0bfe97cd1 kilosort       default_kiloso neuropixels_1. 3bae3f7153f030 linear_interp  2019-09-18 14: 3        0              3             good          
40bfa03a7a7018 0              2019-09-11 14: det_144c54937f a1c6a0bfe97cd1 kilosort       default_kiloso neuropixels_1. 3bae3f7153f030 linear_interp  2019-09-18 14: 4        0              2             good          
40bfa03a7a7018 0              2019-09-11 14: det_144c54937f a1c6a0bfe97cd1 kilosort       default_kiloso neuropixels_1. 3bae3f71

## 4) For animal named "24321", pick one session, fetch the x, y position of the tracked rigid body of that animal. Bonus: plot it

## 5) For the same animal/session in (4), from each probe, identify the unit with highest spike count

## 6) For those 2 units from (5), fetch the spike times, x, and y positions associated with each spike

## 7) Select any ***neurologger*** recording, fetch the LFP at each tetrode